# Transformers Interpret Multiclass Classification Example

In [18]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

## Import Industry Classification Mode
This finetuned model by @sampathkethineedi uses a distilbert base to predict the professional industry a text is referring to. 

In [19]:
tokenizer = AutoTokenizer.from_pretrained("sampathkethineedi/industry-classification")
model = AutoModelForSequenceClassification.from_pretrained(
    "sampathkethineedi/industry-classification"
)

Let's explore the classes, there are 62 unique classes. Many of these are overlapping/related industries such as __Health Care Equipment__ and __Health Care Supplies__.

In [20]:
model.config.id2label

{0: 'Advertising',
 1: 'Aerospace & Defense',
 2: 'Apparel Retail',
 3: 'Apparel, Accessories & Luxury Goods',
 4: 'Application Software',
 5: 'Asset Management & Custody Banks',
 6: 'Auto Parts & Equipment',
 7: 'Biotechnology',
 8: 'Building Products',
 9: 'Casinos & Gaming',
 10: 'Commodity Chemicals',
 11: 'Communications Equipment',
 12: 'Construction & Engineering',
 13: 'Construction Machinery & Heavy Trucks',
 14: 'Consumer Finance',
 15: 'Data Processing & Outsourced Services',
 16: 'Diversified Metals & Mining',
 17: 'Diversified Support Services',
 18: 'Electric Utilities',
 19: 'Electrical Components & Equipment',
 20: 'Electronic Equipment & Instruments',
 21: 'Environmental & Facilities Services',
 22: 'Gold',
 23: 'Health Care Equipment',
 24: 'Health Care Facilities',
 25: 'Health Care Services',
 26: 'Health Care Supplies',
 27: 'Health Care Technology',
 28: 'Homebuilding',
 29: 'Hotels, Resorts & Cruise Lines',
 30: 'Human Resource & Employment Services',
 31: 'IT Co

Import __SequenceClassificationExplainer__ from transformers interpret. This class should work with most if not all language models with a sequence classification head from the transformers package. 


In [21]:
from transformers_interpret import SequenceClassificationExplainer

In [22]:
sample_text = """
Stocks ended a choppy session mixed as investors digested a host of corporate earnings results and considered policymakers’ next moves to support the still virus-stricken economy.
The S&P 500 shook off earlier declines to narrowly eke out a record closing high.The Dow ended a tick below its recent record closing level."""

In [23]:
multiclass_explainer = SequenceClassificationExplainer(text=sample_text, model=model, tokenizer=tokenizer)

In [24]:
# call the exlplainer
attributions = multiclass_explainer()

In [25]:
# seems to be an appropriate prediction
multiclass_explainer.predicted_class_name

'Investment Banking & Brokerage'

In [26]:
# look the the raw word attributions
attributions.word_attributions

[('BOS_TOKEN', 0.0),
 ('Stocks', -0.004456741467730228),
 ('ended', 0.13193411668768767),
 ('a', 0.056411549659932295),
 ('choppy', -0.012832653695455687),
 ('session', -0.13861638195151932),
 ('mixed', 0.046772014985904914),
 ('as', 0.004562625031740414),
 ('investors', -0.02196100971228579),
 ('digested', 0.3463117699963133),
 ('a', -0.15007237744066892),
 ('host', -0.2723240045062653),
 ('of', 0.021451277365530166),
 ('corporate', 0.19881984653293971),
 ('earnings', 0.08638392319925758),
 ('results', 0.20302100667530776),
 ('and', 0.21544563171165798),
 ('considered', 0.16101577787308416),
 ('policymakers’', 0.026943727362501158),
 ('next', -0.07904325843448648),
 ('moves', 0.23334843054630117),
 ('to', 0.037448353157484925),
 ('support', -0.04450222286965038),
 ('the', 0.04380056195019436),
 ('still', 0.15665500295281176),
 ('virus-stricken', -0.04790115963194608),
 ('economy.', 0.07189596512282359),
 ('The', 0.02121638714028611),
 ('S&P', -0.2329136487423103),
 ('500', 0.047506292

## Visualizating Explanations 
With a single call to the `visualize()` method we get a nice inline display of what inputs are causing the activations to fire that led to this prediction. **Note the alogirthm used to calcualte attributions are Layer Integreated Gradients to read more about them click [here](https://captum.ai/docs/algorithms)**

In [27]:
multiclass_explainer.visualize()

## Explaining The Same Text For A Different Class 
Lets say we think this text could also fall somewhat under the class of __Asset Management & Custody Banks__ If we want it is also possible to get an explantion/attributions for the text with that class

In [28]:
attributions = multiclass_explainer(class_name="Asset Management & Custody Banks")

In [29]:
# look the the raw word attributions
attributions.word_attributions

[('BOS_TOKEN', 0.0),
 ('Stocks', -0.013076093437139466),
 ('ended', 0.21332438787938696),
 ('a', -0.06501736140537122),
 ('choppy', -0.05377650405681654),
 ('session', -0.13618844604494268),
 ('mixed', 0.0524596274309297),
 ('as', 0.11133689370437873),
 ('investors', 0.029009141422987397),
 ('digested', 0.35082490371802794),
 ('a', -0.03137677849547649),
 ('host', -0.0824256120985008),
 ('of', -0.12824966411558325),
 ('corporate', 0.1932059939797767),
 ('earnings', 0.14453150173440635),
 ('results', 0.1749567986388768),
 ('and', 0.18457682852945065),
 ('considered', 0.16502232930342287),
 ('policymakers’', 0.05724637287425622),
 ('next', 0.04470654954517382),
 ('moves', 0.4196253362060768),
 ('to', 0.0772446951479946),
 ('support', 0.06417823661238761),
 ('the', 0.08633115904130688),
 ('still', 0.1391353262265933),
 ('virus-stricken', 0.06091509898963959),
 ('economy.', 0.12192535620726684),
 ('The', -0.09255242912925289),
 ('S&P', -0.07162788259214121),
 ('500', 0.10373776462033504),


The results are close to the first visualization, a good sign that the model is generalizing well for both of these related classes

In [30]:
multiclass_explainer.visualize()

What if we get attributions for class that makes no sense in this context such as __Restaurants__?


In [31]:
attributions = multiclass_explainer(class_name="Restaurants")

There isn't much to this prediction, it is worth nothing however that the words "choppy" had a more positive impact in this instance which seems plausible given the industry. 

In [32]:
multiclass_explainer.visualize()